In [16]:
import subprocess
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("DataProcessing").getOrCreate()

command = "hadoop fs -ls /datalake/silver/stagging | awk '{print $NF}'"
dir_names = subprocess.check_output(command, shell=True).decode().split('\n')
print(dir_names)
array_silver =[]
for dir_name in dir_names:
    if (dir_name != "items" and dir_name != "") :
        array_silver.append(dir_name)
        command = f"hadoop fs -ls {dir_name}" + "| awk '{print $NF}'"
        file_names = subprocess.check_output(command, shell=True).decode().split('\n')
        for name in file_names:
            if ".parquet" in name:
                #print(name)
                pass
                
print(array_silver)
df = spark.read.parquet(*array_silver)
df.show(5)
df.count()

['items', '/datalake/silver/stagging/17062024_190437', '']
['/datalake/silver/stagging/17062024_190437']
+--------------------+-------------------+-----------+-------------------+-------------+---------+----------+------------+-----------+------------+----------+--------+----------+----------------+--------------------+-----------------+-------------+
|             commune|current_temperature|customer_id|               date|  employee_id|event_day|event_hour|event_minute|event_month|event_second|event_year|latitude| longitude|    neighborhood|            order_id|quantity_products|sale_point_id|
+--------------------+-------------------+-----------+-------------------+-------------+---------+----------+------------+-----------+------------+----------+--------+----------+----------------+--------------------+-----------------+-------------+
|Comuna 8 - Villa ...|               22.9|       1016|15/06/2024 16:35:11|Employee_0_wd|       15|        16|          35|          6|          11| 

627

In [17]:
from pyspark.sql.window import Window
from datetime import datetime
import pytz
from pyspark.sql.functions import sum as _sum, max as _max, concat_ws, col, udf, to_timestamp, year, month, dayofmonth, hour, minute, second, to_date, date_format, row_number, desc
from pyspark.sql.types import StructType, StructField, IntegerType, StringType

In [18]:
df_seller_aggregated = df.groupBy("event_day","event_month", "employee_id") \
    .agg(_sum("quantity_products").alias("total_sales")).orderBy(col("total_sales").desc())
df_seller_aggregated.show()

+---------+-----------+--------------+-----------+
|event_day|event_month|   employee_id|total_sales|
+---------+-----------+--------------+-----------+
|       15|          6|Employee_69_wd|        296|
|       15|          6|Employee_73_wd|        284|
|       15|          6|Employee_18_wd|        252|
|       15|          6| Employee_5_wd|        236|
|       15|          6|Employee_80_wd|        220|
|       15|          6|Employee_78_wd|        215|
|       15|          6|Employee_59_wd|        212|
|       15|          6|Employee_44_wd|        208|
|       15|          6|Employee_99_wd|        205|
|       16|          6|Employee_63_wd|        203|
|       15|          6|Employee_11_wd|        200|
|       15|          6| Employee_9_wd|        192|
|       15|          6|Employee_15_wd|        188|
|       15|          6| Employee_0_wd|        188|
|       15|          6|Employee_71_wd|        188|
|       15|          6|Employee_91_wd|        183|
|       15|          6|Employee

In [19]:
# Agrupar los datos por fecha y tienda para calcular el total de ventas en esa tienda
df_store_aggregated = df.groupBy("sale_point_id") \
    .agg(_sum("quantity_products").alias("total_store")).orderBy(col("total_store").desc())
df_store_aggregated.show()

+-------------+-----------+
|sale_point_id|total_store|
+-------------+-----------+
|           63|        400|
|           73|        375|
|           69|        369|
|           78|        361|
|            5|        343|
|           80|        318|
|           27|        310|
|           50|        304|
|           43|        303|
|           71|        286|
|           10|        285|
|           23|        282|
|           47|        279|
|           91|        274|
|           30|        265|
|           72|        265|
|           15|        264|
|           24|        257|
|           52|        254|
|           18|        252|
+-------------+-----------+
only showing top 20 rows



In [20]:
# Agrupar los datos por fecha y tienda para calcular el total de ventas en esa tienda
df_store_aggregated = df.groupBy("commune") \
    .agg(_sum("quantity_products").alias("total_commune")).orderBy(col("total_commune").desc())
df_store_aggregated.show()

+--------------------+-------------+
|             commune|total_commune|
+--------------------+-------------+
|Comuna 14 - El Po...|         2413|
| Comuna 5 - Castilla|         2217|
|  Comuna 7 - Robledo|         1878|
|Comuna 8 - Villa ...|         1834|
| Comuna 3 - Manrique|         1491|
|   Comuna 16 - Belén|         1360|
|Comuna 9 - Buenos...|         1179|
| Comuna 4 - Aranjuez|         1116|
|Comuna 15 - Guayabal|          985|
|Comuna 11 - Laure...|          965|
|Comuna 10 - La Ca...|          947|
|  Comuna 1 - Popular|          758|
|Comuna 12 - La Am...|          599|
|Comuna 6 - Doce d...|          474|
|Comuna 13 - San J...|          244|
|Comuna 2 - Santa ...|          228|
+--------------------+-------------+



In [21]:
# Filtrar para obtener solo el vendedor con más ventas cada día
df_top_seller = df_seller_aggregated.groupBy("event_day","event_month") \
                .max("total_sales")
df_top_seller.show()

+---------+-----------+----------------+
|event_day|event_month|max(total_sales)|
+---------+-----------+----------------+
|       15|          6|             296|
|       17|          6|              50|
|       16|          6|             203|
+---------+-----------+----------------+

